# Data Processing process (python version)

In [12]:
import graph_visualisation as gv

from IPython.display import Image

def save_png_and_svg_then_render_dot_graph(dot_source, filename_no_ext):
    tmp_file = open(filename_no_ext, "w", encoding="utf-8")
    tmp_file.write(dot_source)
    tmp_file.close()
    
    gv.render_graph(filename_no_ext, filetype='svg')
    gv.render_graph(filename_no_ext, filetype='png')
    
    image = Image(filename=filename_no_ext + ".png", embed=True, format='png')

    return image

In [ ]:
## Graph process

## Load Test data as pandas dataframe

In [211]:
import pandas as pd
data = pd.read_csv('./data/Test_RNAseq_Data.csv')

In [212]:
### Sample id
data.Run

0    SRR7191189
1    SRR7191190
2    SRR7191191
3    SRR7191192
4    SRR7191193
5    SRR7191194
6    SRR7191195
7    SRR7191196
Name: Run, dtype: object

In [ ]:
!./fastqc 

## Download data by using fasterq-dump (sra)

In [ ]:
# Tuturial (fasterq-dump): https://github.com/ncbi/sra-tools/wiki/HowTo:-fasterq-dump

In [17]:
import subprocess
for i in data.Run:
    subprocess.run([
        "fasterq-dump",
    #     "--split-3",
        i
    ])
    print(i)

SRR7191189
SRR7191190
SRR7191191
SRR7191192
SRR7191193
SRR7191194
SRR7191195
SRR7191196


In [ ]:
# TOBETESTED: TO parallelise this process

In [21]:
import subprocess

template = 'fasterq-dump {}'

args = list(data.Run)
# args = ['SRR6666666','SRR6666667']
processes = []

for arg in args:
    command = template.format(*[arg])
#     print(command)
    process = subprocess.Popen(command, shell=True)
    processes.append(process)

# Collect statuses
output = [p.wait() for p in processes]

fasterq-dump SRR6666666
fasterq-dump SRR6666667


## Run FastQC to test the quality of those datasets

In [26]:
!fastqc --help


            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

So if your input has one file, the output should be ID.fastqc and ID.fastqc.zip. If you unzip the file, you will get a summary.txt file which indicates the results. Also, there is a results txt file for all the data that used for generating graphs.

In [63]:
!mkdir testout

In [ ]:
outputdir = 'testout/'

In [80]:
fastqfile_1 = " ".join(list(data.Run + '_1' + '.fastq')) + " -t "+ str(len(data.Run))
fastqfile_2 = " ".join(list(data.Run + '_2' + '.fastq')) + " -t "+ str(len(data.Run))
files = [fastqfile_1,fastqfile_2]

In [100]:
import subprocess

template = 'fastqc -o ./testout/ {}'

args = list(data.Run + '_2' + '.fastq') # run for _1 as well

processes = []

for arg in args:
    command = template.format(*[arg])
#     print(command)
    process = subprocess.Popen(command, shell=True)
    processes.append(process)

# Collect statuses
output = [p.wait() for p in processes]

fastqc -o ./testout/ SRR7191189_2.fastq
fastqc -o ./testout/ SRR7191190_2.fastq
fastqc -o ./testout/ SRR7191191_2.fastq
fastqc -o ./testout/ SRR7191192_2.fastq
fastqc -o ./testout/ SRR7191193_2.fastq
fastqc -o ./testout/ SRR7191194_2.fastq
fastqc -o ./testout/ SRR7191195_2.fastq
fastqc -o ./testout/ SRR7191196_2.fastq


It takes forever, personally think it might not be a good choice to do this in python

In [156]:
from IPython.display import IFrame

IFrame(src='testout/SRR7191189_2_fastqc.html', width=700, height=600)

### Get all the fastQC output filenames

In [111]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('testout') if isfile(join('testout', f))]

In [121]:
!mkdir summary

In [125]:
identify = []

for i in onlyfiles:
    if i[-1] == 'p':
        subprocess.run([
            "unzip",'./testout/'+ i, '-d', './summary'
        ])
        identify.append(i[0:-4])

In [547]:
cat SRR7191189_1_fastqc/fastqc_data.txt

##FastQC	0.11.9
>>Basic Statistics	pass
#Measure	Value
Filename	SRR7191189_1.fastq
File type	Conventional base calls
Encoding	Sanger / Illumina 1.9
Total Sequences	68452760
Sequences flagged as poor quality	0
Sequence length	25-100
%GC	50
>>END_MODULE
>>Per base sequence quality	pass
#Base	Mean	Median	Lower Quartile	Upper Quartile	10th Percentile	90th Percentile
1	40.059466440798005	41.0	41.0	41.0	37.0	41.0
2	40.03664943531861	41.0	41.0	41.0	37.0	41.0
3	40.06407043339085	41.0	41.0	41.0	37.0	41.0
4	40.04433609397196	41.0	41.0	41.0	37.0	41.0
5	40.034695708398026	41.0	41.0	41.0	37.0	41.0
6	40.04169625885063	41.0	41.0	41.0	37.0	41.0
7	40.02143649138471	41.0	41.0	41.0	37.0	41.0
8	40.01407240555385	41.0	41.0	41.0	37.0	41.0
9	40.034389409572384	41.0	41.0	41.0	37.0	41.0
10-11	40.012003343619746	41.0	41.0	41.0	37.0	41.0
12-13	40.001757504007145	41.0	41.0	41.0	37.0	41.0
14-15	39.97554508393818	41.0	41.0	41.0	37.0	41.0
16-17	39.903069795870906	41.0	41.0	41.0	37.0	41.0
18-19	39.86958782231717	41.0

In [539]:
def find_overseq(filename, withoutAT = True):
    
    f = open(filename,'r')
    txtread = f.read()
    sequence = []
    in_code_section = False
    # id filename.split('/')[2]
    
    for line in txtread.split('\n'):
        line = line.strip()
        
        if line == '#Sequence	Count	Percentage	Possible Source':
            in_code_section = True
            continue
        
        if line.startswith(">>END_MODULE"):
            in_code_section = in_code_section*-1
            
        if in_code_section and not line.startswith(">>END_MODULE"):
            line.split('\t')
        
            tmp_seq = line.split('\t')[0]
            
            if (tmp_seq == 'T'*50 or tmp_seq == 'A'*50) and withoutAT:
                continue
            else:
                sequence.append(tmp_seq)
        
        if in_code_section == -1:
            return sequence

    return sequence

In [213]:
Basic_stat = None
df = []


for i in identify:

    filename = './summary/' + i + '/summary.txt'
    tmp = pd.read_csv(filename, delimiter='\t')
    new_row = [j for j in tmp.PASS]
    new_row.insert(0,i[0:-7])
    df.append(new_row)
    
    if Basic_stat is None:

        Basic_stat = list(tmp.iloc[:,1])
        Basic_stat.insert(0,'id')

In [502]:
filename

'./summary/SRR7191194_2_fastqc/fastqc_data.txt'

In [540]:
sequences = []

for i in identify:

    filename = './summary/' + i + '/fastqc_data.txt'
    sequences.append(find_overseq(filename))

In [541]:
sequences

[[],
 [],
 [],
 ['GCACACGTCTGAACTCCAGTCACCCTATCCTATATCTCGTATGCCGTCTT'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['GCACACGTCTGAACTCCAGTCACTATAGCCTATATCTCGTATGCCGTCTT'],
 [],
 [],
 [],
 [],
 []]

In [214]:
df = pd.DataFrame(df,columns=Basic_stat)

In [542]:
df['sequences_withoutAT'] = sequences 

In [549]:
# onlyfiles = [f for f in listdir('testout') if isfile(join('testout', f))]

In [567]:
def find_adapter(filename):
    
    with open(filename,'r') as f:
        txtread = f.read()
    sequence = []
    in_code_section = False
    # id filename.split('/')[2]
    
    for line in txtread.split('\n'):
        line = line.strip()
        
        if line == "#Position	Illumina Universal Adapter	Illumina Small RNA 3' Adapter	Illumina Small RNA 5' Adapter	Nextera Transposase Sequence	SOLID Small RNA Adapter":
            in_code_section = True
            continue
        
        if line.startswith(">>END_MODULE"):
            in_code_section = in_code_section*-1
            
        if in_code_section and not line.startswith(">>END_MODULE"):
            line.split('\t')
            sequence.append(line.split('\t'))
        
        if in_code_section == -1:
            return sequence[-1][1:6]

    return 'Cannot find anything'

In [568]:
adapters = []

for i in identify:

    filename = './summary/' + i + '/fastqc_data.txt'
    adapter = find_adapter(filename)
    adapters.append([float(i)>1 for i in adapter])


In [569]:
df['adapters'] = adapters

In [570]:
adapter_names = "Illumina Universal Adapter	Illumina Small RNA 3' Adapter	Illumina Small RNA 5' Adapter	Nextera Transposase Sequence	SOLID Small RNA Adapter".split('\t')

In [571]:
adapter_names

['Illumina Universal Adapter',
 "Illumina Small RNA 3' Adapter",
 "Illumina Small RNA 5' Adapter",
 'Nextera Transposase Sequence',
 'SOLID Small RNA Adapter']

In [599]:
# set adapter dictionary (can be found in fastqc adapter_list.txt) could be generated automatically
dictionary = {'Illumina Universal Adapter':'AGATCGGAAGAG',
              "Illumina Small RNA 3' Adapter":'TGGAATTCTCGG',
              "Illumina Small RNA 5' Adapter":'GATCGTCGGACT',
              'Nextera Transposase Sequence':'CTGTCTCTTATA',
              'SOLID Small RNA Adapter':'CGCCTTGGCCGT',
             }

In [600]:
adapter_seqs = []

for adapter_list in adapters:
   
    adapter_seq = []
    
    for i,adapter_name in zip(adapter_list,adapter_names):
        
        if i == True:
            adapter_seq.append(dictionary[adapter_name])
            
    adapter_seqs.append(adapter_seq)

In [603]:
df['adapter_seqs'] = adapter_seqs

## Run cutadapter to cut off the low quality data

In [207]:
!cutadapt --help

cutadapt version 2.8

Copyright (C) 2010-2019 Marcel Martin <marcel.martin@scilifelab.se>

cutadapt removes adapter sequences from high-throughput sequencing reads.

Usage:
    cutadapt -a ADAPTER [options] [-o output.fastq] input.fastq

For paired-end reads:
    cutadapt -a ADAPT1 -A ADAPT2 [options] -o out1.fastq -p out2.fastq in1.fastq in2.fastq

Replace "ADAPTER" with the actual sequence of your 3' adapter. IUPAC wildcard
characters are supported. All reads from input.fastq will be written to
output.fastq with the adapter sequence removed. Adapter matching is
error-tolerant. Multiple adapter sequences can be given (use further -a
options), but only the best-matching adapter will be removed.

Input may also be in FASTA format. Compressed input and output is supported and
auto-detected from the file name (.gz, .xz, .bz2). Use the file name '-' for
standard input/output. Without the -o option, output is sent to standard output.

Citation:

Marcel Martin. Cutadapt removes adapter seque

In [ ]:
#TODO: test cutadapter

In [299]:
!mkdir cutpair

In [384]:
input1 = i + '_1.fastq'
input2 = i + '_2.fastq'

df.loc[df.id == i[0:12]]

,id,Per base sequence quality,Per sequence quality scores,Per base sequence content,Per sequence GC content,Per base N content,Sequence Length Distribution,Sequence Duplication Levels,Overrepresented sequences,Adapter Content,sequences
15,SRR7191194_2,PASS,PASS,PASS,PASS,PASS,WARN,FAIL,WARN,PASS,[TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...


In [589]:
def parameter_generation(sequence1, parameter = '-a'):
    adapters1 = []
    if len(sequence1.iloc[0]) is not 0:
        for j in sequence1.iloc[0]:
            adapters1.append(parameter)
            adapters1.append(j)
    return adapters1

In [607]:
tmpdir = 'cutpair/'

for i in data.Run:
    
    input1 = i + '_1'
    input2 = i + '_2'   
    input_dir1 = i + '_1.fastq'
    input_dir2 = i + '_2.fastq'
    output_dir1 = tmpdir + i + '_1.fastq'
    output_dir2 = tmpdir + i + '_2.fastq'
    
    sequence1 = df.loc[df.id == input1].sequences_withoutAT # change here for AAA
    sequence2 = df.loc[df.id == input2].sequences_withoutAT
    
    adapter_seq1 = df.loc[df.id == input1].adapter_seqs
    adapter_seq2 = df.loc[df.id == input1].adapter_seqs
    
    # -a and -A for input1 and input2
    over_seq1 = parameter_generation(sequence1)
    over_seq2 = parameter_generation(sequence2,'-A')

    # -a and -A for adapters
    adapter1 = parameter_generation(adapter_seq1)
    adapter2 = parameter_generation(adapter_seq2,'-A')
    
    # command generation
    command = ['cutadapt', '-q', '20'] # cut down the low quality reads
    command.extend(over_seq1)
    command.extend(over_seq2)
    command.extend(adapter1)
    command.extend(adapter2)

    command.extend(['-o', output_dir1, # output_dir for input1
                    '-p', output_dir2, # output_dir for input2
                    '-j', '80',
                    input_dir1,
                    input_dir2])
    print(" ".join(command))



cutadapt -q 20 -a GCACACGTCTGAACTCCAGTCACTATAGCCTATATCTCGTATGCCGTCTT -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191189_1.fastq -p cutpair/SRR7191189_2.fastq -j 80 SRR7191189_1.fastq SRR7191189_2.fastq
cutadapt -q 20 -a GCACACGTCTGAACTCCAGTCACCCTATCCTATATCTCGTATGCCGTCTT -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191190_1.fastq -p cutpair/SRR7191190_2.fastq -j 80 SRR7191190_1.fastq SRR7191190_2.fastq
cutadapt -q 20 -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191191_1.fastq -p cutpair/SRR7191191_2.fastq -j 80 SRR7191191_1.fastq SRR7191191_2.fastq
cutadapt -q 20 -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191192_1.fastq -p cutpair/SRR7191192_2.fastq -j 80 SRR7191192_1.fastq SRR7191192_2.fastq
cutadapt -q 20 -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191193_1.fastq -p cutpair/SRR7191193_2.fastq -j 80 SRR7191193_1.fastq SRR7191193_2.fastq
cutadapt -q 20 -a AGATCGGAAGAG -A AGATCGGAAGAG -o cutpair/SRR7191194_1.fastq -p cutpair/SRR7191194_2.fastq -j 80 SRR7191194_1.fastq SRR71

In [606]:
command

['cutadapt',
 '-q',
 '20',
 '-a',
 'GCACACGTCTGAACTCCAGTCACTATAGCCTATATCTCGTATGCCGTCTT',
 '-a',
 True,
 '-a',
 False,
 '-a',
 False,
 '-a',
 False,
 '-a',
 False,
 '-A',
 True,
 '-A',
 False,
 '-A',
 False,
 '-A',
 False,
 '-A',
 False,
 '-o',
 'cutpair/SRR7191189_1.fastq',
 '-p',
 'cutpair/SRR7191189_2.fastq',
 '-j',
 '80',
 'SRR7191189_1.fastq',
 'SRR7191189_2.fastq']

In [586]:
tmpdir = 'cutpair/'

for i in data.Run:
    
    input1 = i + '_1'
    input2 = i + '_2'   
    input_dir1 = i + '_1.fastq'
    input_dir2 = i + '_2.fastq'
    output_dir1 = tmpdir + i + '_1.fastq'
    output_dir2 = tmpdir + i + '_2.fastq'
    
    sequence1 = df.loc[df.id == input1].sequences_withoutAT # change here for AAA
    sequence2 = df.loc[df.id == input2].sequences_withoutAT

    adapter_seq1 = df.loc[df.id == input1].adapters
    adapter_seq2 = df.loc[df.id == input1].adapters
    
    adapters1 = []
    if len(sequence1.iloc[0]) is not 0:
        for j in sequence1.iloc[0]:
            adapters1.append('-a')
            adapters1.append(j)

    adapters2 = []
    if len(sequence2.iloc[0]) is not 0:
        for j in sequence2.iloc[0]:
            adapters2.append('-A')
            adapters2.append(j)
    
    
    command = ['cutadapt', '-q', '20'] # cut down the low quality reads
    command.extend(adapters1)
    command.extend(adapters2)

    command.extend(['-o', output_dir1, # output_dir for input1
                    '-p', output_dir2, # output_dir for input2
                    '-j', '80',
                    input_dir1,
                    input_dir2])
    print(" ".join(command))

#     subprocess.run(command)

cutadapt -q 20 -a GCACACGTCTGAACTCCAGTCACTATAGCCTATATCTCGTATGCCGTCTT -o cutpair/SRR7191189_1.fastq -p cutpair/SRR7191189_2.fastq -j 80 SRR7191189_1.fastq SRR7191189_2.fastq
cutadapt -q 20 -a GCACACGTCTGAACTCCAGTCACCCTATCCTATATCTCGTATGCCGTCTT -o cutpair/SRR7191190_1.fastq -p cutpair/SRR7191190_2.fastq -j 80 SRR7191190_1.fastq SRR7191190_2.fastq
cutadapt -q 20 -o cutpair/SRR7191191_1.fastq -p cutpair/SRR7191191_2.fastq -j 80 SRR7191191_1.fastq SRR7191191_2.fastq
cutadapt -q 20 -o cutpair/SRR7191192_1.fastq -p cutpair/SRR7191192_2.fastq -j 80 SRR7191192_1.fastq SRR7191192_2.fastq
cutadapt -q 20 -o cutpair/SRR7191193_1.fastq -p cutpair/SRR7191193_2.fastq -j 80 SRR7191193_1.fastq SRR7191193_2.fastq
cutadapt -q 20 -o cutpair/SRR7191194_1.fastq -p cutpair/SRR7191194_2.fastq -j 80 SRR7191194_1.fastq SRR7191194_2.fastq
cutadapt -q 20 -o cutpair/SRR7191195_1.fastq -p cutpair/SRR7191195_2.fastq -j 80 SRR7191195_1.fastq SRR7191195_2.fastq
cutadapt -q 20 -o cutpair/SRR7191196_1.fastq -p cutpair/SRR

In [301]:
!cutadapt -q 20 -o cutpair/out.1.fastq -p cutpair/out.2.fastq SRR7191192_1.fastq SRR7191192_2.fastq

This is cutadapt 2.8 with Python 3.7.6
Command line parameters: -q 20 -o cutpair/out.1.fastq -p cutpair/out.2.fastq SRR7191192_1.fastq SRR7191192_2.fastq
Processing reads on 1 core in paired-end mode ...
[------->8   ] 00:12:10    77,263,058 reads  @      9.5 µs/read;   6.34 M reads/minute
Finished in 731.00 s (9 us/read; 6.34 M reads/minute).

=== Summary ===

Total read pairs processed:         77,263,058
  Read 1 with adapter:                       0 (0.0%)
  Read 2 with adapter:                       0 (0.0%)
Pairs written (passing filters):    77,263,058 (100.0%)

Total basepairs processed: 15,452,610,605 bp
  Read 1: 7,726,305,531 bp
  Read 2: 7,726,305,074 bp
Quality-trimmed:             201,314,526 bp (1.3%)
  Read 1:    41,944,144 bp
  Read 2:   159,370,382 bp
Total written (filtered):  15,251,296,079 bp (98.7%)
  Read 1: 7,684,361,387 bp
  Read 2: 7,566,934,692 bp


In [298]:
!cutadapt -q 20 -o cut/SRR7191192_2.fastq SRR7191192_2.fastq

This is cutadapt 2.8 with Python 3.7.6
Command line parameters: -q 20 -o cut/SRR7191192_2.fastq SRR7191192_2.fastq
Processing reads on 1 core in single-end mode ...
[---->8      ] 00:05:06    77,263,058 reads  @      4.0 µs/read;  15.12 M reads/minute
Finished in 306.68 s (4 us/read; 15.12 M reads/minute).

=== Summary ===

Total reads processed:              77,263,058
Reads with adapters:                         0 (0.0%)
Reads written (passing filters):    77,263,058 (100.0%)

Total basepairs processed: 7,726,305,074 bp
Quality-trimmed:             159,370,382 bp (2.1%)
Total written (filtered):  7,566,934,692 bp (97.9%)


## STAR

In [230]:
args_1 = list(data.Run + '_1' + '.fastq')
args_2 = list(data.Run + '_2' + '.fastq')
args_1 = args_1[0:-1]
args_2 = args_2[0:-1]

In [ ]:
STAR --runThreadN 10 --genomeDir ./genome --readFilesIn ./data/SRR7191196_1.fastq ./data/SRR7191196_2.fastq --outFileNamePrefix ./test_sample/ --outSAMtype BAM SortedByCoordinate --sjdbGTFfile gencode.v33.primary_assembly.annotation.gtf --sjdbOverhang 100 --twopassMode Basic --outWigType bedGraph --outWigStrand Stranded


In [232]:
genomedir = '/mnt/Tank/junfan/project/test1/genome'
GTFdir = '/mnt/Tank/junfan/project/test1/gencode.v33.primary_assembly.annotation.gtf'

In [269]:
!mkdir STAROUTPUT

In [498]:
args_1

['SRR7191189_1.fastq',
 'SRR7191190_1.fastq',
 'SRR7191191_1.fastq',
 'SRR7191192_1.fastq',
 'SRR7191193_1.fastq',
 'SRR7191194_1.fastq',
 'SRR7191195_1.fastq']

In [503]:
import subprocess

template = 'STAR --runThreadN 80 --genomeDir ' + genomedir + ' --readFilesIn {} {} --outFileNamePrefix ./{} --outSAMtype BAM SortedByCoordinate --sjdbGTFfile '+ GTFdir + ' --sjdbOverhang 100 --twopassMode Basic --outWigType bedGraph --outWigStrand Stranded'

parent_dir = "/mnt/Tank/junfan/project/test"

commands = []

readfiledir = './'

# subprocess.run(['ulimit', '-n', '4096'])

with open('star-command.sh','w+') as f:
    
    f.write('# set limit\n')
    f.write('ulimit -n 4096\n')
    
    for input1, input2 in zip(args_1,args_2):

        # create folders

    #     output_folder = 'STAROUTPUT/' + input1[0:10]
    #     path = os.path.join(parent_dir, output_folder)
    #     try: 
    #         os.mkdir(path) 
    #     except OSError as error: 
    #         print(error)     

        command = template.format(*[readfiledir+input1, readfiledir+input2, output_folder])
        f.write('\n\n# id:'+ input1[0:10] + '\n\n')
        f.write(command)
#         subprocess.run(command.split(' '))


In [504]:
command

'STAR --runThreadN 80 --genomeDir /mnt/Tank/junfan/project/test1/genome --readFilesIn ./SRR7191195_1.fastq ./SRR7191195_2.fastq --outFileNamePrefix ./STAROUTPUT/SRR7191195 --outSAMtype BAM SortedByCoordinate --sjdbGTFfile /mnt/Tank/junfan/project/test1/gencode.v33.primary_assembly.annotation.gtf --sjdbOverhang 100 --twopassMode Basic --outWigType bedGraph --outWigStrand Stranded'

In [505]:
!bash star-command.sh 

Jan 24 10:41:36 ..... started STAR run
Jan 24 10:41:36 ..... loading genome
Jan 24 10:41:54 ..... processing annotations GTF
Jan 24 10:42:06 ..... inserting junctions into the genome indices
^C


In [265]:
import subprocess

template = 'STAR --runThreadN 10 --genomeDir ' + genomedir + ' --readFilesIn {} {} --outFileNamePrefix ./{} --outSAMtype BAM SortedByCoordinate --sjdbGTFfile '+ GTFdir + ' --sjdbOverhang 100 --twopassMode Basic --outWigType bedGraph --outWigStrand Stranded'

parent_dir = "/mnt/Tank/junfan/project/test"

processes = []

for input1, input2 in zip(args_1,args_2):
    
    # create folders
    
#     output_folder = 'STAROUTPUT/' + input1[0:10]
#     path = os.path.join(parent_dir, output_folder)
#     try: 
#         os.mkdir(path) 
#     except OSError as error: 
#         print(error)     
    
    command = template.format(*[input1, input2, output_folder])
    process = subprocess.Popen(command, shell=True)
    processes.append(process)

# Collect statuses
output = [p.wait() for p in processes]

## Counts to Different Expression

There are two tools to use: **edgeR** and **DESeq2**
```{R}
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("edgeR")
```

In order to use edgeR, you will also need to install limma.
```
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("limma")
```

In [ ]:
library(limma)
library(edgeR)

In [ ]:
STAR --runThreadN 79 --runMode genomeGenerate --genomeDir ./ \--genomeFastaFiles ./Homo_sapiens.GRCh38.dna.primary_assembly.fa 


In [ ]:
STAR \--runThreadN 12 --genomeDir ~/star/genome/ \--sjdbGTFfile ~/star/Homo_sapiens.GRCh38.79.gtf --sjdbOverhang 100 \--readFilesIn SRR7191196_1.fastq SRR7191196_2.fastq \--readFilesCommand zcat

In [ ]:
STAR --runThreadN 20 --readFilesIn tq --quantMode TranscriptomeSAM --outSAMtype BAM SortedByCoordinate --outFileNamePrefix /home/fanyc/RNA-seq/STAR/23 --outFilterType BySJout --outFilterMultimapNmax 20 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --chimSegmentMin 20

In [ ]:
STAR --runThreadN 20 --readFilesIn SRR7191196_1.fastq SRR7191196_2.fastq --quantMode TranscriptomeSAM --outSAMtype BAM SortedByCoordinate --outFileNamePrefix ./output --outFilterType BySJout --outFilterMultimapNmax 20 --alignSJoverhangMin 8 --alignSJDBoverhangMin 1 --outFilterMismatchNmax 999 --outFilterMismatchNoverLmax 0.04 --alignIntronMin 20 --alignIntronMax 1000000 --alignMatesGapMax 1000000 --chimSegmentMin 20

In [ ]:
fasterq-dump --split-3 SRR7191190

In [ ]:
import subprocess
process = subprocess.Popen(['echo', 'More output'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
stdout, stderr